In [0]:
%pip install -U -qqqq backoff databricks-openai uv databricks-agents mlflow==3.9.0rc0 dspy
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


#Pull the Traces with Feedback

We need the feedback you put in during the review app to align judges. 

First we will set up a list of traces with your feedback

In [0]:
import json
from pathlib import Path
import mlflow
from mlflow.genai.datasets import get_dataset

CONFIG = json.loads(Path("config/dc_assistant.json").read_text())

# Extract configuration variables
EXPERIMENT_ID = CONFIG["mlflow"]["experiment_id"]
DATASET_NAME = CONFIG["evaluation"]["dataset_name"]
JUDGE_MODEL = CONFIG["llm"]["judge_model"]
REFLECTION_MODEL = CONFIG["prompt_registry"]["reflection_model"]

# Set the MLflow experiment
mlflow.set_experiment(experiment_id=EXPERIMENT_ID)

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/2517718719552044', creation_time=1768690316954, experiment_id='2517718719552044', last_update_time=1768803175855, lifecycle_stage='active', name=('/Users/austin.choi@databricks.com/GenAI/mlflow updates/AC updates '
 'dc-assistant-agent_experiment'), tags={'mlflow.databricks.managed_evals.experiment_permissions_check': '',
 'mlflow.databricks.review_app.experiment_permissions_check': '',
 'mlflow.experiment.sourceName': '/Users/austin.choi@databricks.com/GenAI/mlflow '
                                 'updates/AC updates '
                                 'dc-assistant-agent_experiment',
 'mlflow.experimentKind': 'genai_development',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.latestTraceEvaluationTimestampMs': '1768882940728',
 'mlflow.ownerEmail': 'austin.choi@databricks.com',
 'mlflow.ownerId': '3275534733162887',
 'product': 'mlflow',
 'purpose': 'football_analysis'}>

In [0]:
traces_for_alignment = mlflow.search_traces(
    experiment_ids=[EXPERIMENT_ID],
    # optionally keep your tag filter, but it’s not sufficient by itself
    filter_string="tag.eval = 'complete'",
    return_type="list",
    max_results=35,  # use 100+ if you can
)

valid_traces = []
for trace in traces_for_alignment:
    feedbacks = trace.search_assessments(name="football_analysis_base")
    has_judge = any(f.source.source_type == "LLM_JUDGE" for f in feedbacks)
    has_human = any(f.source.source_type == "HUMAN" for f in feedbacks)
    if has_judge and has_human:
        valid_traces.append(trace)

print("candidate:", len(traces_for_alignment), "valid:", len(valid_traces))


/home/spark-e76be798-c6ef-4b90-9bbb-36/.ipykernel/3562/command-5175179492058528-3059450067:1: FutureWarning: Parameter 'experiment_ids' is deprecated. Please use 'locations' instead.
  traces_for_alignment = mlflow.search_traces(


candidate: 35 valid: 31


#Judge Alignment

In this tutorial, we will demonstrate two Judge Alignment optimizers: SIMBA and MemAlign. 

###MemAlign 
is a lightweight, dual-memory framework designed to align LLM judges with human experts by efficiently learning from a small amount of natural language feedback, offering a faster and cheaper alternative to traditional prompt engineering or fine-tuning. The system uses Semantic Memory for general principles and Episodic Memory for specific examples, allowing for rapid adaptation and showing visible improvement with as few as 2-10 examples. This approach delivers competitive or better quality than state-of-the-art prompt optimizers at up to 100x lower latency and 10x lower cost, and it is now the default optimization algorithm in MLflow's align() method in MLflow 3.9+

###SIMBA 
(Stochastic Introspective Mini-Batch Ascent) is a DSPy prompt-optimization method that iteratively improves an LLM’s prompts by evaluating changes on mini-batches with a target metric. It uses a stochastic hill-climbing loop that proposes prompt edits (including instruction rewrites and/or few-shot demonstrations) and keeps the variants that score better. Its “introspective” step leverages the LLM to analyze failures and generate corrective guidance, reducing manual prompt tuning for complex tasks. It was the default optimizer for mlflow.align() in MLflow 3.8 or below and can still be used today.

#Load Judge we want to Align

In [0]:
# Load the base judge from the evaluation notebook and define configuration parameters
from mlflow.genai.scorers import get_scorer

# SIMBA configuration parameters
LIKERT_MIN = 1.0  # Minimum Likert scale value
LIKERT_MAX = 5.0  # Maximum Likert scale value
SIMBA_BATCH_SIZE = 8  # Number of examples per SIMBA optimization step
SIMBA_MAX_DEMOS = 0  # Maximum few-shot demos (0 recommended for situations where exact matches on responses are not really possible)
SIMBA_VERBOSE = True  # Enable verbose logging for SIMBA optimization

# Load the base football analysis judge from the evaluation notebook
# Reference: https://mlflow.org/docs/latest/api_reference/python_api/mlflow.genai.html#mlflow.genai.scorers.get_scorer
judge_name = CONFIG.get("evaluation", {}).get("judge_name", "football_analysis_base")
football_analysis_judge = get_scorer(name=judge_name)

# Define the aligned judge name (can be customized in config if needed)
ALIGNED_JUDGE_NAME = CONFIG.get("evaluation", {}).get("aligned_judge_name", "football_analysis_judge_align")

print(f"Loaded base judge from evaluation notebook: {judge_name}")
print(f"Aligned judge name: {ALIGNED_JUDGE_NAME}")
print(f"SIMBA config: batch_size={SIMBA_BATCH_SIZE}, max_demos={SIMBA_MAX_DEMOS}, Likert range=[{LIKERT_MIN}, {LIKERT_MAX}]")


Loaded base judge from evaluation notebook: football_analysis_base
Aligned judge name: football_analysis_judge_align
SIMBA config: batch_size=8, max_demos=0, Likert range=[1.0, 5.0]


#SIMBA Implementation

Below is a basic Judge Alignment using the default SIMBA implementation from MLflow. 

#Regular SIMBA Optimization

In [0]:
import logging
from mlflow.genai.judges.optimizers import SIMBAAlignmentOptimizer
from statistics import mean
from typing import Any, Callable, List

from mlflow.genai.judges.base import AlignmentOptimizer, Judge
from mlflow.entities.trace import Trace

logging.getLogger("mlflow.genai.judges.optimizers.simba").setLevel(logging.DEBUG)

print(f'Initial Judge Text /n {football_analysis_judge.instructions}')

aligned_judge_basic = football_analysis_judge.align(
    traces=valid_traces,
    optimizer=SIMBAAlignmentOptimizer(model=REFLECTION_MODEL),
)

2026/01/19 03:37:56 INFO mlflow.genai.judges.optimizers.simba: Starting SIMBA optimization with 31 examples (set logging to DEBUG for detailed output)
2026/01/19 03:37:56 INFO dspy.teleprompt.simba: Starting batch 1 of 8.
2026/01/19 03:37:56 INFO dspy.teleprompt.simba: Sampling program trajectories on 10 examples x 6 samples.


Initial Judge Text /n Evaluate if the response in {{ outputs }} appropriately analyzes the available data and provides an actionable recommendation the question in {{ inputs }}. The response should be accurate, contextually relevant, and give a strategic advantage to the  person making the request. Your grading criteria should be:  1: Completely unacceptable. Incorrect data interpretation or no recommendations 2: Mostly unacceptable. Irrelevant or spurious feedback or weak recommendations provided with minimal strategic advantage 3: Somewhat acceptable. Relevant feedback provided with some strategic advantage 4: Mostly acceptable. Relevant feedback provided with strong strategic advantage 5 Completely acceptable. Relevant feedback provided with excellent strategic advantage
Processed 60 / 60 examples: 100%|██████████| 60/60 [00:12<00:00,  4.64it/s]

2026/01/19 03:38:09 INFO dspy.teleprompt.simba: Batch 1: Baseline mini-batch score: 0.05

2026/01/19 03:38:09 INFO dspy.teleprompt.simba: Batch 1: Processing bucket #1, with max score 1.0, max-to-min gap 1.0, and max-to-avg gap 0.5.
2026/01/19 03:38:09 INFO dspy.teleprompt.simba: Batch 1: Invoking strategy: append_a_rule
2026/01/19 03:38:09 INFO dspy.teleprompt.simba_utils: Skipping rule generation as good score True is at or below the 10th percentile *or* bad score False is at or above the 90th percentile.
2026/01/19 03:38:09 INFO dspy.teleprompt.simba: 

2026/01/19 03:38:09 INFO dspy.teleprompt.simba: Batch 1: Processing bucket #2, with max score 0.0, max-to-min gap 0.0, and max-to-avg gap 0.0.
2026/01/19 03:38:09 INFO dspy.teleprompt.simba: Batch 1: Invoking strategy: append_a_rule
2026/01/19 03:38:09 INFO dspy.teleprompt.simba_utils: Skipping rule generation as good score False is at or below the 10th percentile *or* bad score False is at or above the 90th percentile.
2026/01/19 03


Processed 70 / 70 examples: 100%|██████████| 70/70 [00:14<00:00,  4.82it/s]

2026/01/19 03:38:24 INFO dspy.teleprompt.simba: Scores after 1 batches: [0.0, 0.0, 0.1, 0.0, 0.0, 0.1, 0.0], Best: 0.1

2026/01/19 03:38:24 INFO dspy.teleprompt.simba: Starting batch 2 of 8.
2026/01/19 03:38:24 INFO dspy.teleprompt.simba: Sampling program trajectories on 10 examples x 6 samples.


...

2026/01/19 03:41:41 INFO dspy.teleprompt.simba: Batch 8: Baseline mini-batch score: 0.05

2026/01/19 03:41:41 INFO dspy.teleprompt.simba: Batch 8: Processing bucket #1, with max score 1.0, max-to-min gap 1.0, and max-to-avg gap 0.8333333333333334.
2026/01/19 03:41:41 INFO dspy.teleprompt.simba: Batch 8: Invoking strategy: append_a_rule
2026/01/19 03:41:41 INFO dspy.teleprompt.simba_utils: Skipping rule generation as good score True is at or below the 10th percentile *or* bad score False is at or above the 90th percentile.
2026/01/19 03:41:41 INFO dspy.teleprompt.simba: 

2026/01/19 03:41:41 INFO dspy.teleprompt.simba: Batch 8: Processing bucket #2, with max score 1.0, max-to-min gap 1.0, and max-to-avg gap 0.6666666666666667.
2026/01/19 03:41:41 INFO dspy.teleprompt.simba: Batch 8: Invoking strategy: append_a_demo_
2026/01/19 03:41:41 INFO dspy.teleprompt.simba_utils: Added 0 demos (one each) across all predictors.
2026/01/19 03:41:41 INFO dspy.teleprompt.simba: 

2026/01/19 03:41:41 I


Processed 70 / 70 examples: 100%|██████████| 70/70 [00:15<00:00,  4.58it/s]

2026/01/19 03:41:56 INFO dspy.teleprompt.simba: Scores after 8 batches: [0.1, 0.0, 0.1, 0.1, 0.2, 0.1, 0.2], Best: 0.2

2026/01/19 03:41:56 INFO dspy.teleprompt.simba: VALIDATION: Evaluating 7 programs on the full trainset.



Processed 217 / 217 examples: 100%|██████████| 217/217 [00:43<00:00,  4.98it/s]

2026/01/19 03:42:40 INFO dspy.teleprompt.simba: Final trainset scores: [0.0967741935483871, 0.03225806451612903, 0.0967741935483871, 0.06451612903225806, 0.0967741935483871, 0.3548387096774194, 0.41935483870967744], Best: 0.41935483870967744 (at index 6)



2026/01/19 03:42:40 INFO mlflow.genai.judges.optimizers.simba: SIMBA optimization completed


In [0]:
print("Original instructions:\n", football_analysis_judge.instructions)
print("\nAligned instructions:\n", aligned_judge_basic.instructions)

Original instructions:
 Evaluate if the response in {{ outputs }} appropriately analyzes the available data and provides an actionable recommendation the question in {{ inputs }}. The response should be accurate, contextually relevant, and give a strategic advantage to the  person making the request. Your grading criteria should be:  1: Completely unacceptable. Incorrect data interpretation or no recommendations 2: Mostly unacceptable. Irrelevant or spurious feedback or weak recommendations provided with minimal strategic advantage 3: Somewhat acceptable. Relevant feedback provided with some strategic advantage 4: Mostly acceptable. Relevant feedback provided with strong strategic advantage 5 Completely acceptable. Relevant feedback provided with excellent strategic advantage

Aligned instructions:
 Evaluate if the response in {{ outputs }} appropriately analyzes the available data and provides an actionable recommendation the question in {{ inputs }}. The response should be accurate, 

In [0]:
from mlflow.genai.judges import make_judge
from mlflow.genai.scorers import (
    Guidelines,
    RelevanceToQuery,
    ScorerSamplingConfig,
    get_scorer
)

register_align_judge_basic = make_judge(
    name=f"{ALIGNED_JUDGE_NAME}_basic",
    instructions=aligned_judge_basic.instructions,
    feedback_value_type=float,
    # model=JUDGE_MODEL,  # Model used to evaluate (from config)
)

try:
    register_aligned_judge_basic = register_align_judge_basic.register(experiment_id=EXPERIMENT_ID)

except ValueError as e:
    msg = str(e)

    if "has already been registered" in msg:
        # Preferred path per the error message: update existing scorer
        register_aligned_judge = register_align_judge_basic.update(
            experiment_id=EXPERIMENT_ID,
            sampling_config=ScorerSamplingConfig(sample_rate=1)
        )
    else:
        raise

print("Registered aligned judge", register_aligned_judge_basic.name)

Registered aligned judge football_analysis_judge_align_basic


###Rerun with SIMBA Optimized Judge

In [0]:
from agent import AGENT
from mlflow.genai import evaluate
from mlflow.genai.datasets import create_dataset, get_dataset
from mlflow.genai.scorers import (
    Guidelines,
    RelevanceToQuery,
    get_scorer,
)

# Compile the judges and rerun the evaluation job

football_language = "The response must use language that is appropriate for professional football players and coaches"
football_language_judge = Guidelines(name="football_language", guidelines=football_language)

scorers = [RelevanceToQuery(), football_language_judge, register_aligned_judge_basic]
# Rerun the evaluation after recreating the judge to better calibrate the agent quality

mlflow.set_experiment(experiment_id=EXPERIMENT_ID)

# Grab all traces from the original eval dataset
eval_dataset = get_dataset(name=DATASET_NAME)

def extract_question(row_input):
    try:
      messages = row_input['request']['input']
      last_message = messages[-1]
      return last_message['content']
    except Exception as e:
      print(e)

df = eval_dataset.to_df()
eval_dataset_records = [
    {
        "inputs": {
            "input": [
                # Pass the EXTRACTED string, not the whole dictionary object
                {"role": "user", "content": extract_question(row)}
            ]
        }
        # Note: "expected" field is optional
    }
    for row in df['inputs'] 
]

print("Executing Evaluation Job")
results = evaluate(
    data=eval_dataset_records,
    predict_fn=lambda input: AGENT.predict({"input": input}),
    scorers=scorers
)

2026/01/19 04:44:01 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset. To disable this check, set the MLFLOW_GENAI_EVAL_SKIP_TRACE_VALIDATION environment variable to True.
2026/01/19 04:44:01 WARNING mlflow.tracing.fluent: Failed to start span predict_stream: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.


Executing Evaluation Job


Evaluating:   0%|          | 0/32 [Elapsed: 00:00, Remaining: ?] 

2026/01/19 04:44:26 WARNING mlflow.genai.judges.instructions_judge: The following parameters were provided but are not used by this judge's instructions: 'expectations'. The judge only uses template variables that appear in the instructions: {'outputs', 'inputs'}
2026/01/19 04:44:46 WARNING mlflow.genai.judges.instructions_judge: The following parameters were provided but are not used by this judge's instructions: 'expectations'. The judge only uses template variables that appear in the instructions: {'outputs', 'inputs'}
2026/01/19 04:44:47 WARNING mlflow.genai.judges.instructions_judge: The following parameters were provided but are not used by this judge's instructions: 'expectations'. The judge only uses template variables that appear in the instructions: {'outputs', 'inputs'}
2026/01/19 04:45:00 WARNING mlflow.genai.judges.instructions_judge: The following parameters were provided but are not used by this judge's instructions: 'expectations'. The judge only uses template variables

#MemAlign Optimizer 

Mlflow's default optimizer

In [0]:
import logging
import os
import dspy
from mlflow.genai.judges.optimizers import MemAlignOptimizer
from statistics import mean
from typing import Any, Callable, List

from mlflow.genai.judges.base import AlignmentOptimizer, Judge
from mlflow.entities.trace import Trace

os.environ["OPENAI_API_KEY"] = ""
dspy.configure(cache=False) 

print(f'Initial Judge Text /n {football_analysis_judge.instructions}')

aligned_judge_memalign = football_analysis_judge.align(
    traces=valid_traces,
    optimizer=MemAlignOptimizer(reflection_lm="databricks:/databricks-gpt-5-2", embedding_model="openai/text-embedding-3-large",) #Databricks not supported right now
)

Initial Judge Text /n Evaluate if the response in {{ outputs }} appropriately analyzes the available data and provides an actionable recommendation the question in {{ inputs }}. The response should be accurate, contextually relevant, and give a strategic advantage to the  person making the request. Your grading criteria should be:  1: Completely unacceptable. Incorrect data interpretation or no recommendations 2: Mostly unacceptable. Irrelevant or spurious feedback or weak recommendations provided with minimal strategic advantage 3: Somewhat acceptable. Relevant feedback provided with some strategic advantage 4: Mostly acceptable. Relevant feedback provided with strong strategic advantage 5 Completely acceptable. Relevant feedback provided with excellent strategic advantage


In [0]:
print("Original instructions:\n", football_analysis_judge.instructions)
print("\nAligned instructions:\n", aligned_judge_memalign.instructions)

Original instructions:
 Evaluate if the response in {{ outputs }} appropriately analyzes the available data and provides an actionable recommendation the question in {{ inputs }}. The response should be accurate, contextually relevant, and give a strategic advantage to the  person making the request. Your grading criteria should be:  1: Completely unacceptable. Incorrect data interpretation or no recommendations 2: Mostly unacceptable. Irrelevant or spurious feedback or weak recommendations provided with minimal strategic advantage 3: Somewhat acceptable. Relevant feedback provided with some strategic advantage 4: Mostly acceptable. Relevant feedback provided with strong strategic advantage 5 Completely acceptable. Relevant feedback provided with excellent strategic advantage

Aligned instructions:
 Evaluate if the response in {{ outputs }} appropriately analyzes the available data and provides an actionable recommendation the question in {{ inputs }}. The response should be accurate, 

In [0]:
from mlflow.genai.judges import make_judge
from mlflow.genai.scorers import (
    Guidelines,
    RelevanceToQuery,
    ScorerSamplingConfig,
    get_scorer
)

register_align_judge_memalign = make_judge(
    name=f"{ALIGNED_JUDGE_NAME}_memalign",
    instructions=aligned_judge_memalign.instructions,
    feedback_value_type=float,
    # model=JUDGE_MODEL,  # Model used to evaluate (from config)
)

try:
    register_aligned_judge_memalign = register_align_judge_memalign.register(experiment_id=EXPERIMENT_ID)
    print("Registered aligned judge", register_aligned_judge_memalign.name)

except ValueError as e:
    msg = str(e)

    if "has already been registered" in msg:
        # Preferred path per the error message: update existing scorer
        register_aligned_judge = register_align_judge_memalign.update(
            experiment_id=EXPERIMENT_ID,
            sampling_config=ScorerSamplingConfig(sample_rate=1)
        )
    else:
        raise



###Run the MemAlign Aligned Judge

In [0]:
from agent import AGENT
from mlflow.genai import evaluate
from mlflow.genai.datasets import create_dataset, get_dataset
from mlflow.genai.scorers import (
    Guidelines,
    RelevanceToQuery,
    get_scorer,
)

# Compile the judges and rerun the evaluation job

football_language = "The response must use language that is appropriate for professional football players and coaches"
football_language_judge = Guidelines(name="football_language", guidelines=football_language)

scorers = [RelevanceToQuery(), football_language_judge, register_align_judge_memalign]
# Rerun the evaluation after recreating the judge to better calibrate the agent quality

mlflow.set_experiment(experiment_id=EXPERIMENT_ID)

# Grab all traces from the original eval dataset
eval_dataset = get_dataset(name=DATASET_NAME)

def extract_question(row_input):
    try:
      messages = row_input['request']['input']
      last_message = messages[-1]
      return last_message['content']
    except Exception as e:
      print(e)

df = eval_dataset.to_df()
eval_dataset_records = [
    {
        "inputs": {
            "input": [
                # Pass the EXTRACTED string, not the whole dictionary object
                {"role": "user", "content": extract_question(row)}
            ]
        }
        # Note: "expected" field is optional
    }
    for row in df['inputs'] 
]

print("Executing Evaluation Job")
results = evaluate(
    data=eval_dataset_records,
    predict_fn=lambda input: AGENT.predict({"input": input}),
    scorers=scorers
)

/databricks/python_shell/lib/dbruntime/autoreload/discoverability/autoreload_discoverability_hook.py:72: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  return orig_warn(*args, **kwargs)
2026/01/19 06:12:49 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.
2026/01/19 06:12:49 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset. To disable this check, set the MLFLOW_GENAI_EVAL_SKIP_TRACE_VALIDATION environment variable to True.
2026/01/19 06:12:49 WARNING mlflow.tracing.fluent: Failed to start span predict_stream: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.


Executing Evaluation Job


Evaluating:   0%|          | 0/32 [Elapsed: 00:00, Remaining: ?] 

2026/01/19 06:13:22 WARNING mlflow.genai.judges.instructions_judge: The following parameters were provided but are not used by this judge's instructions: 'expectations'. The judge only uses template variables that appear in the instructions: {'outputs', 'inputs'}
2026/01/19 06:13:31 WARNING mlflow.genai.judges.instructions_judge: The following parameters were provided but are not used by this judge's instructions: 'expectations'. The judge only uses template variables that appear in the instructions: {'outputs', 'inputs'}
2026/01/19 06:13:31 WARNING mlflow.genai.judges.instructions_judge: The following parameters were provided but are not used by this judge's instructions: 'expectations'. The judge only uses template variables that appear in the instructions: {'outputs', 'inputs'}
2026/01/19 06:13:37 WARNING mlflow.genai.judges.instructions_judge: The following parameters were provided but are not used by this judge's instructions: 'expectations'. The judge only uses template variables

#Likert SIMBA Optimizer

MLflow allows you to build or create a new optimizer using `AlignmentOptimizer`. To demonstrate this, we have created a Likert-aware SIMBA optimizer below to improve its optimization 

In [0]:
# Likert-aware SIMBA optimizer: single-cell implementation

import logging
from statistics import mean
from typing import Any, Callable, List

from mlflow.genai.judges.base import AlignmentOptimizer, Judge
from mlflow.entities.trace import Trace
from mlflow.genai.judges.optimizers import SIMBAAlignmentOptimizer as _BaseSIMBA

# Use configuration parameters for Likert scale
def _to_float_maybe(x: Any) -> float | None:
    try:
        return float(x)
    except Exception:
        return None

def likert_agreement_metric(example: Any, prediction: Any) -> float:
    """
    Likert agreement metric:
        score = 1 - |llm - human| / (LIKERT_MAX - LIKERT_MIN)

    Reads from:
      - Human label: example._store["result"]
      - LLM/judge score: prediction._store["result"]
    """
    logger = logging.getLogger("dspy.teleprompt.simba")

    human = None
    llm = None

    # Primary: read from example._store / prediction._store
    ex_store = getattr(example, "_store", None)
    if isinstance(ex_store, dict) and "result" in ex_store:
        human = _to_float_maybe(ex_store["result"])

    pred_store = getattr(prediction, "_store", None)
    if isinstance(pred_store, dict) and "result" in pred_store:
        llm = _to_float_maybe(pred_store["result"])

    # Fallbacks
    if human is None:
        for key in ("human_score", "human_value", "label", "target", "score", "y"):
            if hasattr(example, key):
                human = _to_float_maybe(getattr(example, key))
                if human is not None:
                    break
            if isinstance(example, dict) and key in example:
                human = _to_float_maybe(example[key])
                if human is not None:
                    break

    if llm is None:
        if isinstance(prediction, dict):
            for k in ("llm_score", "value", "score", "rating", "label", "y_hat"):
                if k in prediction:
                    llm = _to_float_maybe(prediction[k])
                    if llm is not None:
                        break
        if llm is None:
            llm = _to_float_maybe(prediction)

    if human is None or llm is None:
        logger.info(
            "LIKERT: missing scores (human=%r, llm=%r) -> 0.0",
            human,
            llm,
        )
        return 0.0

    # Clamp to configured Likert range
    human = max(LIKERT_MIN, min(LIKERT_MAX, human))
    llm = max(LIKERT_MIN, min(LIKERT_MAX, llm))

    score = max(0.0, 1.0 - abs(llm - human) / (LIKERT_MAX - LIKERT_MIN))
    return score


class LikertSIMBAAlignmentOptimizer(AlignmentOptimizer):
    """Unified optimizer: injects Likert metric, batch size, max_demos, and optional verbose logging.

    Uses configuration parameters from the config cell above.
    """

    def __init__(
        self,
        model: str,
        batch_size: int = 10,
        max_demos: int = 0,
        metric_fn: Callable[[Any, Any], float] = None,
        verbose: bool = False,
    ):
        self.model = model
        self.batch_size = batch_size
        self.max_demos = max_demos
        self.metric_fn = metric_fn
        self.verbose = verbose

    # ---- Internal helpers for verbose logging ----
    class _BatchScoreAggregator:
        def __init__(self):
            self.all_batches: List[List[float]] = []
            self.current: List[float] = []
            self.batch_idx: int = 0

        def start_batch(self):
            if self.current:
                self._log_current_summary()
                self.all_batches.append(self.current)
            self.current = []
            self.batch_idx += 1

        def add(self, score: float):
            if isinstance(score, (int, float)):
                self.current.append(float(score))

        def end(self):
            if self.current:
                self._log_current_summary()
                self.all_batches.append(self.current)
                self.current = []
            all_flat = [s for batch in self.all_batches for s in batch]
            if all_flat:
                best = max(all_flat)
                batches_n = len(self.all_batches)
                logging.getLogger("dspy.teleprompt.simba").info(
                    "Scores after %d batches: %s, Best: %s",
                    batches_n,
                    [round(mean(b), 3) if b else 0.0 for b in self.all_batches],
                    round(best, 3),
                )

        def _log_current_summary(self):
            lg = logging.getLogger("dspy.teleprompt.simba")
            if not self.current:
                return
            mx = max(self.current)
            mn = min(self.current)
            avg = mean(self.current)
            lg.info(
                "Processing bucket #%d, with max score %s, max-to-min gap %s, and max-to-avg gap %s.",
                self.batch_idx if self.batch_idx else 1,
                round(mx, 3),
                round(mx - mn, 3),
                round(mx - avg, 3),
            )

    class _SIMBABatchLogHandler(logging.Handler):
        def __init__(self, aggregator: "LikertSIMBAAlignmentOptimizer._BatchScoreAggregator"):
            super().__init__()
            self.aggregator = aggregator

        def emit(self, record: logging.LogRecord):
            msg = record.getMessage()
            if "Starting batch" in msg and "of" in msg:
                self.aggregator.start_batch()

    def _wrap_metric_for_logging(self, metric_fn: Callable[[Any, Any], float]):
        aggregator = self._BatchScoreAggregator()

        def logged_metric(example, prediction):  
            score = metric_fn(example, prediction)
            aggregator.add(score)
            return score

        batch_handler = self._SIMBABatchLogHandler(aggregator)
        simba_logger = logging.getLogger("dspy.teleprompt.simba")
        simba_utils_logger = logging.getLogger("dspy.teleprompt.simba_utils")
        simba_logger.setLevel(logging.INFO)
        simba_utils_logger.setLevel(logging.INFO)
        if all(not isinstance(h, LikertSIMBAAlignmentOptimizer._SIMBABatchLogHandler) for h in simba_logger.handlers):
            simba_logger.addHandler(batch_handler)
        return logged_metric, aggregator, simba_logger, batch_handler

    def align(self, judge: Judge, traces: list[Trace]) -> Judge:
        import dspy.teleprompt.simba as dsimba

        # Choose metric function
        metric_fn = self.metric_fn if self.metric_fn is not None else likert_agreement_metric
        logging.getLogger("dspy.teleprompt.simba").info(
            "Using SIMBA metric_fn=%s",
            getattr(metric_fn, "__name__", repr(metric_fn)),
        )
        
        # Optionally wrap metric for verbose logging
        aggregator = None
        simba_logger = None
        batch_handler = None
        if self.verbose:
            metric_fn, aggregator, simba_logger, batch_handler = self._wrap_metric_for_logging(metric_fn)

        # Patch DSPy SIMBA init to inject our parameters
        original_init = dsimba.SIMBA.__init__
        batch_size = self.batch_size
        max_demos = self.max_demos

        def patched_init(self_, *args, **kwargs): 
            # Force our settings
            logging.getLogger("dspy.teleprompt.simba").info(
                "Patched SIMBA.__init__: forcing metric_fn=%s, bsize=%s, max_demos=%s",
                getattr(metric_fn, "__name__", repr(metric_fn)),
                batch_size,
                max_demos,
            )

            kwargs["metric"] = metric_fn
            kwargs["bsize"] = batch_size
            kwargs["max_demos"] = max_demos

            return original_init(self_, *args, **kwargs)

        dsimba.SIMBA.__init__ = patched_init
        try:
            base = _BaseSIMBA(model=self.model)
            result = base.align(judge=judge, traces=traces)
        finally:
            dsimba.SIMBA.__init__ = original_init
            if aggregator is not None:
                aggregator.end()
            if simba_logger is not None and batch_handler is not None:
                try:
                    simba_logger.removeHandler(batch_handler)
                except Exception:
                    pass
        return result

print("Likert SIMBA optimizer loaded successfully")


Likert SIMBA optimizer loaded successfully


###Run Optimization with LikertSIMBAOptimizer

This will take a few minutes

In [0]:
logging.getLogger("mlflow.genai.judges.optimizers.simba").setLevel(logging.DEBUG)

print(f'Initial Judge Text /n {football_analysis_judge.instructions}')

likert_optimizer = LikertSIMBAAlignmentOptimizer(
    model=REFLECTION_MODEL,
    batch_size=6,
    max_demos=0,
    verbose=True
)

aligned_judge = football_analysis_judge.align(
    traces=valid_traces,
    optimizer=likert_optimizer,
)

2026/01/19 03:52:34 INFO dspy.teleprompt.simba: Using SIMBA metric_fn=likert_agreement_metric
2026/01/19 03:52:34 INFO dspy.teleprompt.simba: Patched SIMBA.__init__: forcing metric_fn=logged_metric, bsize=6, max_demos=0
2026/01/19 03:52:34 INFO mlflow.genai.judges.optimizers.simba: Starting SIMBA optimization with 31 examples (set logging to DEBUG for detailed output)
2026/01/19 03:52:34 INFO dspy.teleprompt.simba: Starting batch 1 of 8.
2026/01/19 03:52:34 INFO dspy.teleprompt.simba: Sampling program trajectories on 6 examples x 6 samples.


Initial Judge Text /n Evaluate if the response in {{ outputs }} appropriately analyzes the available data and provides an actionable recommendation the question in {{ inputs }}. The response should be accurate, contextually relevant, and give a strategic advantage to the  person making the request. Your grading criteria should be:  1: Completely unacceptable. Incorrect data interpretation or no recommendations 2: Mostly unacceptable. Irrelevant or spurious feedback or weak recommendations provided with minimal strategic advantage 3: Somewhat acceptable. Relevant feedback provided with some strategic advantage 4: Mostly acceptable. Relevant feedback provided with strong strategic advantage 5 Completely acceptable. Relevant feedback provided with excellent strategic advantage
Processed 36 / 36 examples: 100%|██████████| 36/36 [00:07<00:00,  4.68it/s]

2026/01/19 03:52:42 INFO dspy.teleprompt.simba: Batch 1: Baseline mini-batch score: 0.4583333333333333

2026/01/19 03:52:42 INFO dspy.teleprompt.simba: Batch 1: Processing bucket #1, with max score 0.75, max-to-min gap 0.0, and max-to-avg gap 0.0.
2026/01/19 03:52:42 INFO dspy.teleprompt.simba: Batch 1: Invoking strategy: append_a_rule
2026/01/19 03:52:42 INFO dspy.teleprompt.simba_utils: Skipping rule generation as good score 0.75 is at or below the 10th percentile *or* bad score 0.75 is at or above the 90th percentile.
2026/01/19 03:52:42 INFO dspy.teleprompt.simba: 

2026/01/19 03:52:42 INFO dspy.teleprompt.simba: Batch 1: Processing bucket #2, with max score 0.5, max-to-min gap 0.0, and max-to-avg gap 0.0.
2026/01/19 03:52:42 INFO dspy.teleprompt.simba: Batch 1: Invoking strategy: append_a_rule


2026/01/19 03:52:55 INFO dspy.teleprompt.simba_utils: Advice for self: If the evaluated response summarizes tendencies (e.g., shotgun pass most common) but does NOT (1) cite or clearly point to specific supporting evidence from the provided data (key plays, counts, sample sizes, notable examples/outliers) and (2) translate that evidence into concrete, situationally deployable coaching decisions (e.g., specific pressure/Cover X calls vs specific formations/personnel, down-and-distance/game-state qualifiers), then you should cap the score at ~3–4, not 5.

If the response contains small-sample caveats yet still makes strong claims without anchoring them (no references to which plays drove EPA, which specific play was the “best outlier,” no table/list of the most important plays), then treat it as ‘somewhat acceptable’ (3) unless it compensates by explicitly highlighting the top key plays and why they matter.

If the user is a coach/strategist and the response provides only generic defensi

Processed 36 / 36 examples: 100%|██████████| 36/36 [00:08<00:00,  4.02it/s]

...

2026/01/19 04:02:21 INFO dspy.teleprompt.simba_utils: Advice for self: When the input question contains “typical play sequence / sequence / script / play-by-play / what do they call first/next / under X minutes”, you must verify the output includes an explicit ordered progression of plays (e.g., “Play 1… Play 2…”, or a sample drive) with situational context like clock/TOs/field position and down & distance. Numbered headings like “1. Formation, 2. Play type priorities” are NOT sufficient if they are just tendencies.

If the output is primarily tendencies (pass rate, personnel frequency, average air yards, route families) and generic recommendations, and does not translate those into an actual series (e.g., “open with quick sideline concept → if incomplete, tempo again; if complete in bounds, call timeout; then take a shot on 1st-and-10; then red-zone/FG sequence”), cap the score around 3 even if the football analysis is otherwise strong.

Do not give a 5 when the answer says “If you wa

Processed 36 / 36 examples: 100%|██████████| 36/36 [00:08<00:00,  4.15it/s]

2026/01/19 04:02:53 INFO dspy.teleprompt.simba: Scores after 8 batches: [0.625, 0.875, 0.5833333333333334, 0.625, 0.5833333333333334, 0.7916666666666666], Best: 0.875

2026/01/19 04:02:53 INFO dspy.teleprompt.simba: VALIDATION: Evaluating 7 programs on the full trainset.



Processed 217 / 217 examples: 100%|██████████| 217/217 [00:46<00:00,  4.70it/s]

2026/01/19 04:03:39 INFO dspy.teleprompt.simba: Final trainset scores: [0.6612903225806451, 0.6935483870967742, 0.8548387096774194, 0.75, 0.7580645161290323, 0.8387096774193549, 0.8145161290322581], Best: 0.8548387096774194 (at index 2)



2026/01/19 04:03:39 INFO mlflow.genai.judges.optimizers.simba: SIMBA optimization completed
2026/01/19 04:03:39 INFO dspy.teleprompt.simba: Processing bucket #8, with max score 1.0, max-to-min gap 1.0, and max-to-avg gap 0.258.
2026/01/19 04:03:39 INFO dspy.teleprompt.simba: Scores after 8 batches: [0.521, 0.76, 0.747, 0.844, 0.75, 0.851, 0.708, 0.742], Best: 1.0


In [0]:
print("Original instructions:\n", football_analysis_judge.instructions)
print("\nAligned instructions:\n", aligned_judge.instructions)

Original instructions:
 Evaluate if the response in {{ outputs }} appropriately analyzes the available data and provides an actionable recommendation the question in {{ inputs }}. The response should be accurate, contextually relevant, and give a strategic advantage to the  person making the request. Your grading criteria should be:  1: Completely unacceptable. Incorrect data interpretation or no recommendations 2: Mostly unacceptable. Irrelevant or spurious feedback or weak recommendations provided with minimal strategic advantage 3: Somewhat acceptable. Relevant feedback provided with some strategic advantage 4: Mostly acceptable. Relevant feedback provided with strong strategic advantage 5 Completely acceptable. Relevant feedback provided with excellent strategic advantage

Aligned instructions:
 Evaluate if the response in {{ outputs }} appropriately analyzes the available data and provides an actionable recommendation the question in {{ inputs }}. The response should be accurate, 

In [0]:
from mlflow.genai.judges import make_judge
from mlflow.genai.scorers import (
    Guidelines,
    RelevanceToQuery,
    ScorerSamplingConfig,
    get_scorer
)

mlflow.set_experiment(experiment_id=EXPERIMENT_ID)

register_align_judge = make_judge(
    name=f"{ALIGNED_JUDGE_NAME}_likert",
    instructions=aligned_judge.instructions,
    feedback_value_type=float,
    # model=JUDGE_MODEL,  # Model used to evaluate (from config)
)

try:
    register_aligned_judge = register_align_judge.register(experiment_id=EXPERIMENT_ID)

except ValueError as e:
    msg = str(e)

    if "has already been registered" in msg:
        # Preferred path per the error message: update existing scorer
        register_aligned_judge = register_align_judge.update(
            experiment_id=EXPERIMENT_ID,
            sampling_config=ScorerSamplingConfig(sample_rate=1)
        )
    else:
        raise

print("Registered aligned judge", register_align_judge.name)

Registered aligned judge football_analysis_judge_align_likert


In [0]:
print(register_aligned_judge.instructions)

Evaluate if the response in {{ outputs }} appropriately analyzes the available data and provides an actionable recommendation the question in {{ inputs }}. The response should be accurate, contextually relevant, and give a strategic advantage to the  person making the request. Your grading criteria should be:  1: Completely unacceptable. Incorrect data interpretation or no recommendations 2: Mostly unacceptable. Irrelevant or spurious feedback or weak recommendations provided with minimal strategic advantage 3: Somewhat acceptable. Relevant feedback provided with some strategic advantage 4: Mostly acceptable. Relevant feedback provided with strong strategic advantage 5 Completely acceptable. Relevant feedback provided with excellent strategic advantage

If the input question asks for a **“typical play sequence”** (keywords: “sequence”, “play-by-play”, “what do they call first/next”, “script”, “with under X minutes”), then you should **check that the output contains an explicit ordered 

###Rerun with Likert SIMBA Optimizer

In [0]:
from agent import AGENT
from mlflow.genai import evaluate
from mlflow.genai.datasets import create_dataset, get_dataset
from mlflow.genai.scorers import (
    Guidelines,
    RelevanceToQuery,
    get_scorer,
)

# Compile the judges and rerun the evaluation job

football_language = "The response must use language that is appropriate for professional football players and coaches"
football_language_judge = Guidelines(name="football_language", guidelines=football_language)

scorers = [RelevanceToQuery(), football_language_judge, register_aligned_judge]
# Rerun the evaluation after recreating the judge to better calibrate the agent quality

mlflow.set_experiment(experiment_id=EXPERIMENT_ID)

# Grab all traces from the original eval dataset
eval_dataset = get_dataset(name=DATASET_NAME)

def extract_question(row_input):
    try:
      messages = row_input['request']['input']
      last_message = messages[-1]
      return last_message['content']
    except Exception as e:
      print(e)

df = eval_dataset.to_df()
eval_dataset_records = [
    {
        "inputs": {
            "input": [
                # Pass the EXTRACTED string, not the whole dictionary object
                {"role": "user", "content": extract_question(row)}
            ]
        }
        # Note: "expected" field is optional
    }
    for row in df['inputs'] 
]

print("Executing Evaluation Job")
results = evaluate(
    data=eval_dataset_records,
    predict_fn=lambda input: AGENT.predict({"input": input}),
    scorers=scorers
)

2026/01/19 04:42:41 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.
2026/01/19 04:42:41 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset. To disable this check, set the MLFLOW_GENAI_EVAL_SKIP_TRACE_VALIDATION environment variable to True.
2026/01/19 04:42:41 WARNING mlflow.tracing.fluent: Failed to start span predict_stream: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.


Executing Evaluation Job


Evaluating:   0%|          | 0/32 [Elapsed: 00:00, Remaining: ?] 

2026/01/19 04:43:02 WARNING mlflow.genai.judges.instructions_judge: The following parameters were provided but are not used by this judge's instructions: 'expectations'. The judge only uses template variables that appear in the instructions: {'outputs', 'inputs'}
2026/01/19 04:43:06 WARNING mlflow.genai.judges.instructions_judge: The following parameters were provided but are not used by this judge's instructions: 'expectations'. The judge only uses template variables that appear in the instructions: {'outputs', 'inputs'}
2026/01/19 04:43:07 WARNING mlflow.genai.judges.instructions_judge: The following parameters were provided but are not used by this judge's instructions: 'expectations'. The judge only uses template variables that appear in the instructions: {'outputs', 'inputs'}
2026/01/19 04:43:08 WARNING mlflow.genai.judges.instructions_judge: The following parameters were provided but are not used by this judge's instructions: 'expectations'. The judge only uses template variables